# Classification Hyperparameter Tuning

- Stephen W. Thomas
- Used for MMA 869, MMAI 869, and GMMA 869

In [1]:
import datetime
print(datetime.datetime.now())

2022-07-14 17:52:43.402142


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.0.2.


In [4]:
# Create the 'out' directory to store output images
import os
if not os.path.exists('out'):
    os.makedirs('out')

# Load Data

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/stepthom/869_course/main/data/GermanCredit.csv')
df['Class'] = df['Class'].map({'Good': 1, 'Bad': 0})
df.head()
X = df.drop(['Class'], axis=1)
y = df[['Class']]

,Duration,Amount,InstallmentRatePercentage,ResidenceDuration,Age,NumberExistingCredits,NumberPeopleMaintenance,Telephone,ForeignWorker,Class,...,OtherInstallmentPlans.Bank,OtherInstallmentPlans.Stores,OtherInstallmentPlans.None,Housing.Rent,Housing.Own,Housing.ForFree,Job.UnemployedUnskilled,Job.UnskilledResident,Job.SkilledEmployee,Job.Management.SelfEmp.HighlyQualified
0,6,1169,4,4,67,2,1,0,1,1,...,0,0,1,0,1,0,0,0,1,0
1,48,5951,2,2,22,1,1,1,1,0,...,0,0,1,0,1,0,0,0,1,0
2,12,2096,2,3,49,1,2,1,1,1,...,0,0,1,0,1,0,0,1,0,0
3,42,7882,2,4,45,1,2,1,1,1,...,0,0,1,0,0,1,0,0,1,0
4,24,4870,3,4,53,2,2,1,1,0,...,0,0,1,0,0,1,0,0,1,0


In [6]:
# Helper function to print out the results of hyperparmater tuning in a nice table.

def cv_results_to_df(cv_results):
    results = pd.DataFrame(list(cv_results['params']))
    #results['mean_fit_time'] = cv_results['mean_fit_time']
    #results['mean_score_time'] = cv_results['mean_score_time']
    #results['mean_train_score'] = cv_results['mean_train_score']
    #results['std_train_score'] = cv_results['std_train_score']
    results['mean_val_score'] = cv_results['mean_test_score']
    #results['std_val_score'] = cv_results['std_test_score']
    results['rank_val_score'] = cv_results['rank_test_score']

    results = results.sort_values(['mean_val_score'], ascending=False)
    return results

# Decision Trees

In [7]:
clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5, random_state=0)

scores = cross_val_score(clf, X, y, cv=10, scoring="accuracy")
print("Mean Accuracy: {:.4f}".format(np.mean(scores)))

Mean Accuracy: 0.6970


In [8]:
clf2 = DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_split=5, random_state=0)

scores = cross_val_score(clf2, X, y, cv=10, scoring="accuracy")
print("Mean Accuracy: {:.4f}".format(np.mean(scores)))

Mean Accuracy: 0.7010


In [9]:
clf3 = DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_split=6, random_state=0)

scores = cross_val_score(clf3, X, y, cv=10, scoring="accuracy")
print("Mean Accuracy: {:.4f}".format(np.mean(scores)))

Mean Accuracy: 0.7040


## Hyperparameter Tuning

### Grid Search

In [10]:
from sklearn.model_selection import GridSearchCV

clf = DecisionTreeClassifier(random_state=42)

params = {'criterion': ('gini', 'entropy'), 
          'splitter': ('best', 'random'), 
          'class_weight': ('balanced', None), 
          'max_depth': [2, 5, 10, 20], 
          'min_samples_leaf': [1, 5, 10],
          'max_features':[0.25, 0.5, 0.75, 1.0]}

search = GridSearchCV(clf, params, scoring='f1_macro', cv=10, verbose=1)
search = search.fit(X, y)

Fitting 10 folds for each of 384 candidates, totalling 3840 fits


In [11]:
search.best_params_

{'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': 10,
 'max_features': 1.0,
 'min_samples_leaf': 1,
 'splitter': 'best'}

In [12]:
search.best_score_

0.6631045623915302

In [13]:
cv_results_to_df(search.cv_results_)

,class_weight,criterion,max_depth,max_features,min_samples_leaf,splitter,mean_val_score,rank_val_score
66,balanced,gini,10,1.00,1,best,0.663105,1
339,None,entropy,10,0.25,5,random,0.662809,2
145,balanced,entropy,10,0.25,1,random,0.659962,3
280,None,gini,20,0.75,10,best,0.658818,4
364,None,entropy,20,0.25,10,best,0.657111,5
...,...,...,...,...,...,...,...,...
196,None,gini,2,0.25,10,best,0.411765,379
194,None,gini,2,0.25,5,best,0.411765,379
288,None,entropy,2,0.25,1,best,0.411765,379
290,None,entropy,2,0.25,5,best,0.411765,379


### Random Search

In [15]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

clf = DecisionTreeClassifier(random_state=42)

params = {"criterion": ["gini", "entropy"],
          "splitter": ["best", "random"],
          "class_weight": ['balanced', None], 
          "max_depth": randint(2, 21),
          "min_samples_leaf": randint(1, 11),
          "max_features": uniform(0.0, 1.0)}

search = RandomizedSearchCV(clf, param_distributions=params, n_iter=1000, scoring='f1_macro', cv=10, verbose=1)
search = search.fit(X, y)

Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


In [16]:
search.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 8,
 'max_features': 0.3305176264926438,
 'min_samples_leaf': 7,
 'splitter': 'random'}

In [18]:
search.best_score_

0.6773351251760114

In [19]:
cv_results_to_df(search.cv_results_)

,class_weight,criterion,max_depth,max_features,min_samples_leaf,splitter,mean_val_score,rank_val_score
142,None,gini,8,0.330518,7,random,0.677335,1
477,None,entropy,18,0.146264,8,best,0.677065,2
839,None,entropy,20,0.740945,8,best,0.671164,3
108,None,entropy,10,0.120987,10,best,0.669481,4
645,None,entropy,20,0.886055,7,best,0.669268,5
...,...,...,...,...,...,...,...,...
738,balanced,entropy,2,0.018126,2,best,0.416271,996
250,None,gini,2,0.042513,1,best,0.411765,997
73,None,gini,2,0.021087,4,best,0.411765,997
304,None,entropy,2,0.170939,10,random,0.411765,997


# Halving Grid Search

In [20]:
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingGridSearchCV

clf = DecisionTreeClassifier(random_state=42)

params = {'criterion': ('gini', 'entropy'), 
          'splitter': ('best', 'random'), 
          'class_weight': ('balanced', None), 
          'max_depth': [2, 5, 10, 20], 
          'min_samples_leaf': [1, 5, 10],
          'max_features':[0.25, 0.5, 0.75, 1.0]}

search = HalvingGridSearchCV(clf, params, scoring='f1_macro', cv=10, verbose=1)
search = search.fit(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


n_iterations: 3
n_required_iterations: 6
n_possible_iterations: 3
min_resources_: 40
max_resources_: 1000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 384
n_resources: 40
Fitting 10 folds for each of 384 candidates, totalling 3840 fits
----------
iter: 1
n_candidates: 128
n_resources: 120
Fitting 10 folds for each of 128 candidates, totalling 1280 fits
----------
iter: 2
n_candidates: 43
n_resources: 360
Fitting 10 folds for each of 43 candidates, totalling 430 fits


In [21]:
search.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 20,
 'max_features': 1.0,
 'min_samples_leaf': 5,
 'splitter': 'random'}

In [22]:
search.best_score_

0.6523930710014034

In [23]:
cv_results_to_df(search.cv_results_)

,class_weight,criterion,max_depth,max_features,min_samples_leaf,splitter,mean_val_score,rank_val_score
285,None,gini,20,1.00,5,random,0.723333,1
261,None,gini,10,1.00,5,random,0.723333,1
237,None,gini,5,1.00,5,random,0.723333,1
303,None,entropy,2,0.75,5,random,0.720000,4
213,None,gini,2,1.00,5,random,0.696667,5
...,...,...,...,...,...,...,...,...
113,balanced,entropy,2,0.75,10,random,0.380000,548
191,balanced,entropy,20,1.00,10,random,0.380000,548
119,balanced,entropy,2,1.00,10,random,0.380000,548
161,balanced,entropy,10,0.75,10,random,0.380000,548
